## Geospatial Python
Setup: https://carpentries-incubator.github.io/geospatial-python/index.html

Instruction: https://carpentries-incubator.github.io/geospatial-python/05-access-data.html

SpatioTemporal Asset Catalog (STAC) Browser

In [5]:
import rioxarray
from pystac_client import Client
from shapely.geometry import Point

In [6]:
api_url = "https://earth-search.aws.element84.com/v1"


client = Client.open(api_url)
collection = "sentinel-2-l2a"  # Sentinel-2, Level 2A, Cloud Optimized GeoTiffs (COGs)


point = Point(4.89, 52.37)  # AMS coordinates

In [7]:
# perform metadata search limited to 10 results

search = client.search(
    collections=[collection],
    intersects=point,
    max_items=10,
)

In [8]:
# show the number of scenes
print(search.matched())

1135


In [9]:
#store the metadata of the search results
items = search.item_collection()

In [10]:
# get the length of items
print(len(items))

10


In [11]:
# loop over the items
for item in items:
    print(item)

<Item id=S2A_31UFU_20240127_0_L2A>
<Item id=S2B_31UFU_20240125_0_L2A>
<Item id=S2B_31UFU_20240122_0_L2A>
<Item id=S2A_31UFU_20240120_0_L2A>
<Item id=S2A_31UFU_20240117_0_L2A>
<Item id=S2B_31UFU_20240112_0_L2A>
<Item id=S2A_31UFU_20240110_0_L2A>
<Item id=S2A_31UFU_20240107_0_L2A>
<Item id=S2B_31UFU_20240105_0_L2A>
<Item id=S2B_31UFU_20240102_0_L2A>


In [12]:
#  inspect the metadata associated with the first item of the search result
item = items[0]
print(item.datetime)
print(item.geometry)
print(item.properties)

2024-01-27 10:46:20.273000+00:00
{'type': 'Polygon', 'coordinates': [[[5.253802482193188, 53.228306912690954], [4.8284658835160466, 52.24839460577159], [6.071664488869862, 52.22257539160585], [6.141754296879459, 53.20819279121764], [5.253802482193188, 53.228306912690954]]]}
{'created': '2024-01-27T16:36:06.073Z', 'platform': 'sentinel-2a', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 7.224917, 'proj:epsg': 32631, 'mgrs:utm_zone': 31, 'mgrs:latitude_band': 'U', 'mgrs:grid_square': 'FU', 'grid:code': 'MGRS-31UFU', 'view:sun_azimuth': 163.83828996157, 'view:sun_elevation': 17.352061312522494, 's2:degraded_msi_data_percentage': 0.0172, 's2:nodata_pixel_percentage': 34.201282, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 0.00836, 's2:cloud_shadow_percentage': 0.537798, 's2:vegetation_percentage': 34.312046, 's2:not_vegetated_percentage': 27.646589, 's2:water_percentage': 30.058819, 's2:unclassified_percentage': 0.208473, 's2:medium

In [14]:
'''
Search for all the available Sentinel-2 scenes in the sentinel-2-l2a collection that satisfy the following criteria: 
- intersect a provided bounding box (use ±0.01 deg in lat/lon from the previously defined point); 
- have been recorded between 20 March 2020 and 30 March 2020; 
- have a cloud coverage smaller than 10%.
'''
bbox = point.buffer(0.01).bounds

search = client.search(
    collections=[collection],
    bbox=bbox,
    datetime="2020-03-20/2020-03-30",
    query=["eo:cloud_cover<15"]
)
print(search.matched())
items = search.item_collection()
items.save_object("search.json") # json file saved alongside notebook

6


Access the assets

In [15]:
assets = items[0].assets  # first item's asset dictionary
print(assets.keys())

dict_keys(['aot', 'blue', 'coastal', 'granule_metadata', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 'rededge2', 'rededge3', 'scl', 'swir16', 'swir22', 'thumbnail', 'tileinfo_metadata', 'visual', 'wvp', 'aot-jp2', 'blue-jp2', 'coastal-jp2', 'green-jp2', 'nir-jp2', 'nir08-jp2', 'nir09-jp2', 'red-jp2', 'rededge1-jp2', 'rededge2-jp2', 'rededge3-jp2', 'scl-jp2', 'swir16-jp2', 'swir22-jp2', 'visual-jp2', 'wvp-jp2'])


In [16]:
# print a minimal description of the available assets:
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

aot: Aerosol optical thickness (AOT)
blue: Blue (band 2) - 10m
coastal: Coastal aerosol (band 1) - 60m
granule_metadata: None
green: Green (band 3) - 10m
nir: NIR 1 (band 8) - 10m
nir08: NIR 2 (band 8A) - 20m
nir09: NIR 3 (band 9) - 60m
red: Red (band 4) - 10m
rededge1: Red edge 1 (band 5) - 20m
rededge2: Red edge 2 (band 6) - 20m
rededge3: Red edge 3 (band 7) - 20m
scl: Scene classification map (SCL)
swir16: SWIR 1 (band 11) - 20m
swir22: SWIR 2 (band 12) - 20m
thumbnail: Thumbnail image
tileinfo_metadata: None
visual: True color image
wvp: Water vapour (WVP)
aot-jp2: Aerosol optical thickness (AOT)
blue-jp2: Blue (band 2) - 10m
coastal-jp2: Coastal aerosol (band 1) - 60m
green-jp2: Green (band 3) - 10m
nir-jp2: NIR 1 (band 8) - 10m
nir08-jp2: NIR 2 (band 8A) - 20m
nir09-jp2: NIR 3 (band 9) - 60m
red-jp2: Red (band 4) - 10m
rededge1-jp2: Red edge 1 (band 5) - 20m
rededge2-jp2: Red edge 2 (band 6) - 20m
rededge3-jp2: Red edge 3 (band 7) - 20m
scl-jp2: Scene classification map (SCL)
swi

In [17]:
# show one metadata value
print(assets["thumbnail"].href)

https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/31/U/FU/2020/3/S2A_31UFU_20200328_1_L2A/thumbnail.jpg


In [18]:
# open nir with the rioxarray library

nir_href = assets["nir"].href
nir = rioxarray.open_rasterio(nir_href)
print(nir)

<xarray.DataArray (band: 1, y: 10980, x: 10980)>
[120560400 values with dtype=uint16]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05 7.098e+05
  * y            (y) float64 5.9e+06 5.9e+06 5.9e+06 ... 5.79e+06 5.79e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  AVERAGE
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0


In [19]:
# save whole image to disk - this may take awhile
nir.rio.to_raster("nir.tif")

In [20]:
# save portion of an image to disk
nir[0,1500:2200,1500:2200].rio.to_raster("nir_subset.tif")

In [21]:
# from https://lpdaac.usgs.gov/products/hlsl30v002/ access connect to the STAC endpoint
'''
Using pystac_client, 
search for all assets of the Landsat 8 collection (HLSL30.v2.0) 
from February to March 2021, 
intersecting the point with longitude/latitute coordinates (-73.97, 40.78) deg.
Visualize an item’s thumbnail (asset key browse).

'''
cmr_api_url = "https://cmr.earthdata.nasa.gov/stac/LPCLOUD"
client = Client.open(cmr_api_url)

# setup search
search = client.search(
    collections=["HLSL30.v2.0"],
    intersects=Point(-73.97, 40.78),
    datetime="2021-02-01/2021-03-30",
) # nasa cmr cloud cover filtering is currently broken: https://github.com/nasa/cmr-stac/issues/239

# retrieve search results
items = search.item_collection()
print(len(items))

5


In [22]:
items_sorted = sorted(items, key=lambda x: x.properties["eo:cloud_cover"]) # sorting and then selecting by cloud cover
item = items_sorted[0]
print(item)

<Item id=HLS.L30.T18TWL.2021039T153324.v2.0>


In [23]:
print(item.assets["browse"].href)

https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-public/HLSL30.020/HLS.L30.T18TWL.2021039T153324.v2.0/HLS.L30.T18TWL.2021039T153324.v2.0.jpg


Public metadata does not mean public data.
Consider getting a free NASA Earthdata account here https://urs.earthdata.nasa.gov/
And creating a netrc file for access here https://git.earthdata.nasa.gov/projects/LPDUR/repos/daac_data_download_python/browse/EarthdataLoginSetup.py

Then using 
*import os<br>
os.environ["GDAL_HTTP_COOKIEFILE"] = "./cookies.txt"<br>
os.environ["GDAL_HTTP_COOKIEJAR"] = "./cookies.txt"*
